In [1]:
# scaricamento dati della f1. 
# preso il mugello siccome hanno giraato nel 2020 
# anche loro ed è il tempio della motogp

import fastf1
import polars as pl
from pathlib import Path


# crea una cartella 'cache' per velocizzare le esecuzioni future)
fastf1.Cache.enable_cache("cache")

session = fastf1.get_session(2020, "Tuscan", "Q")
session.load()


driver = "LEC"
lap = session.laps.pick_driver(driver).pick_fastest()
telemetry = lap.get_telemetry()

df = pl.DataFrame(
    telemetry[["Time", "Speed", "RPM", "nGear", "Throttle", "Brake", "DRS"]]
)

# scale dei dati con rpm maggiori per "simulare" di più le moto
df = df.with_columns(
    [
        (pl.col("Time").dt.total_milliseconds() / 1000).alias("Time_Sec"),
        (pl.col("RPM") * 1.1).alias("RPM"),
    ]
)

parquet_name = "telemetry_mugello_f1_2020.parquet"

file_path = Path.cwd() / "data"
df.write_parquet(file_path)

print(f"Dati reali Mugello salvati in: {parquet_name}")
print(f"Pilota: {driver} | Tempo: {lap['LapTime']}")
print(df.head())

NotADirectoryError: Cache directory does not exist! Please check for typos or create it first.